In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# AERONEFS

In [2]:
dfa = pd.read_csv('data/aeronefs_2023-11-07.csv')

display(dfa.shape)

count_none = dfa[dfa['ref_aero'].str.startswith('None')].shape[0]
print(f'Nbre aero qui commence par None :{count_none}')

count_duplicates = dfa.duplicated(subset=['ref_aero']).sum()
print(f'Nbre duplica aero : {count_duplicates}')

#calcul si aberration de date de maintenance
dfa['end_maint'] = pd.to_datetime(dfa['end_maint'])
dfa['last_maint'] = pd.to_datetime(dfa['last_maint'])

dfa['date_diff'] = (dfa['end_maint'] - dfa['last_maint']).dt.days

display(dfa)


(232, 6)

Nbre aero qui commence par None :22
Nbre duplica aero : 0


,ref_aero,type_model,debut_service,last_maint,en_maintenance,end_maint,date_diff
0,B747_0195,B747,2008-12-15,2023-11-05,False,NaT,NaN
1,A320_4466,A320,2017-08-18,2023-11-04,False,NaT,NaN
2,None_3415,NaN,2007-11-08,2023-11-02,False,NaT,NaN
3,CRJ900_6891,CRJ900,2000-04-30,2023-10-30,False,NaT,NaN
4,A321_2784,A321,2014-02-18,2023-11-04,False,NaT,NaN
...,...,...,...,...,...,...,...
227,E170_2289,E170,2011-12-24,2023-11-06,True,2023-11-07 14:13:00,1.0
228,CRJ900_0271,CRJ900,2009-04-21,2023-11-06,True,2023-11-13 14:13:00,7.0
229,B737_6105,B737,2000-07-27,2023-11-06,True,2023-11-07 14:13:00,1.0
230,E175_0488,E175,2016-04-22,2023-10-26,False,NaT,NaN


# COMPOSANTS

In [2]:
dfc = pd.read_csv('data/composants_2023-11-07.csv')

display(dfc.shape)

unique_desc_values = dfc['desc'].unique()
print(unique_desc_values)


(12257, 6)

["Train d'atterrissage avant" 'Empennage horizontal'
 'Boîte noire (enregistreur de vol)' 'Dispositifs de levage (flaps)'
 'Ordinateur de bord' 'Équipement de navigation par satellite (GPS)'
 'Réacteur droit' 'Porte cargo' "Train d'atterrissage principal droit"
 'Aileron gauche' 'Volet de bord de fuite gauche' 'Siège copilote'
 'Antenne radar' "Système d'extinction d'incendie du moteur"
 "Système d'éclairage de piste" 'Gouvernail de direction'
 "Éclairage d'urgence" "Train d'atterrissage principal gauche"
 'Capteurs de vitesse du vent' 'Générateur électrique'
 'Caméra de surveillance externe' 'Spoilers' "Système d'annonce publique"
 'Système hydraulique principal' 'Aileron droit'
 "Dispositif d'atterrissage d'urgence (comme les toboggans)"
 'Système anti-givre' 'Système de divertissement en vol'
 "Système d'éclairage de cabine" 'Radar météorologique'
 "Système de conditionnement d'air" 'Système de communication radio'
 'Autopilote' 'Détecteurs de fumée en cabine'
 'Système de gestion d

# DEGRADATION

In [4]:
# Créer un dossier 'deg' s'il n'existe pas en chemin relatif pour stocker les fichiers csv (à ne pas faire sur Colab)
import os
if not os.path.exists('data/deg'):
    os.makedirs('data/deg')
import shutil
if not os.path.exists('data/deg/degradations_2023-11-07.csv'):
    shutil.move('data/degradations_2023-11-07.csv', 'data/deg/degradations_2023-11-07.csv')

start_date = '2023-11-07'
today = pd.Timestamp.today().strftime('%Y-%m-%d')

date_range = pd.date_range(start=start_date, end=today, freq='D')
df_each_date = []
df_each_date.append(pd.read_csv('data/deg/degradations_2023-11-07.csv'))

for date in date_range:
    link = f'http://sc-e.fr/docs/degradations_{date.strftime("%Y-%m-%d")}.csv'
    
    try :
        df = pd.read_csv(link)
        df_each_date.append(df)
    
        export = os.path.join('data/deg', f'degradations_{date.strftime("%Y-%m-%d")}.csv')
        df.to_csv(export, index=False)
    except:
        print(f"Le fichier {link} n'existe pas")
        continue

dfd = pd.concat(df_each_date)

display(dfd.shape)
display(dfd.head())

Le fichier http://sc-e.fr/docs/degradations_2023-11-07.csv n'existe pas
Le fichier http://sc-e.fr/docs/degradations_2024-01-04.csv n'existe pas


(11277, 6)

,ref_deg,linked_aero,compo_concerned,usure_cumulée,measure_day,need_replacement
0,D009254,B767_0657,SPO16-014,21.70125,2023-11-07,False
1,D006897,B747_0910,DIS15-246,17.75015,2023-11-07,False
2,D007309,A350_1948,REA1-683,16.31745,2023-11-07,False
3,D000831,B747_4916,EQU23-377,19.24685,2023-11-07,False
4,D001545,E175_0964,TRA21-634,17.66480,2023-11-07,False


In [14]:
dfd_A321_6216 = dfd[dfd['aero'] == "A321_6216"]

# LOGS VOLS

In [5]:
# Créer un dossier 'deg' s'il n'existe pas en chemin relatif pour stocker les fichiers csv (à ne pas faire sur Colab)
import os
if not os.path.exists('data/logs_vols'):
    os.makedirs('data/logs_vols')
import shutil
if not os.path.exists('data/logs_vols/logs_vols_2023-11-07.csv'):
    shutil.move('data/logs_vols_2023-11-07.csv', 'data/logs_vols/logs_vols_2023-11-07.csv')

start_date = '2023-11-07'
today = pd.Timestamp.today().strftime('%Y-%m-%d')

date_range = pd.date_range(start=start_date, end=today, freq='D')
df_each_date = []
df_each_date.append(pd.read_csv('data/logs_vols/logs_vols_2023-11-07.csv'))

for date in date_range:
    link = f'http://sc-e.fr/docs/logs_vols_{date.strftime("%Y-%m-%d")}.csv'
    
    try :
        df = pd.read_csv(link)
        df_each_date.append(df)
    
        export = os.path.join('data/logs_vols', f'logs_vols_{date.strftime("%Y-%m-%d")}.csv')
        df.to_csv(export, index=False)
    except:
        print(f"Le fichier {link} n'existe pas")
        continue

dflv = pd.concat(df_each_date)

display(dflv.shape)
display(dflv.head())

Le fichier http://sc-e.fr/docs/logs_vols_2023-11-07.csv n'existe pas
Le fichier http://sc-e.fr/docs/logs_vols_2024-01-04.csv n'existe pas


(5488, 6)

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant
0,V03656986,CRJ900_5773,2023-11-07 14:13:00,3.9,"{'temp': '15°C', 'pressure': '1039 hPa', 'vibr...",0
1,V07726467,CRJ900_0428,2023-11-07 14:13:00,7.7,"{'temp': '-32°C', 'pressure': '1092 hPa', 'vib...",0
2,V04561239,B777_2551,2023-11-07 14:13:00,5.5,"{'temp': '-32°C', 'pressure': '1099 hPa', 'vib...",0
3,V03775756,None_5980,2023-11-07 14:13:00,9.1,"{'temp': '30°C', 'pressure': '1020 hPa', 'vibr...",0
4,V07191233,A321_3449,2023-11-07 14:13:00,10.0,"{'temp': '-50°C', 'pressure': '913 hPa', 'vibr...",1


In [ ]:
dflv_A321_6216 = dflv[dflv['aero'] == "A321_6216"]

# DF all

In [6]:

# Renommer les colonnes pour avoir le même nom de colonne dans chaque df
dataframes = [dfa, dfc, dfd, dflv]

for df in dataframes:
    if 'aero' not in df.columns:
        if 'ref_aero' in df.columns:
            df.rename(columns={'ref_aero': 'aero'}, inplace=True)
        elif 'linked_aero' in df.columns:
            df.rename(columns={'linked_aero': 'aero'}, inplace=True)
        elif 'aero_linked' in df.columns:
            df.rename(columns={'aero_linked': 'aero'}, inplace=True)
    if 'compo' not in df.columns:
        if 'ref_compo' in df.columns:
            df.rename(columns={'ref_compo': 'compo'}, inplace=True)
        elif 'compo_concerned' in df.columns:
            df.rename(columns={'compo_concerned': 'compo'}, inplace=True)

common_columns = ['aero','compo']
# Merge les df à partir des colonnes communes
df_all = pd.merge(dfc, dfd, on=common_columns)
df_all = pd.merge(df_all, dfa, on='aero')
df_all = pd.merge(df_all, dflv, on='aero')

In [7]:
count_unique_aero = df_all['aero'].nunique()
print(f"Number of unique values in 'aero' column: {count_unique_aero}")


Number of unique values in 'aero' column: 232


In [8]:
count_unique_aero = df_all[df_all['desc'].str.startswith('Boît')]['aero'].nunique()
print(f"Number of unique values in 'aero' column for 'desc' starting with 'boîte noire': {count_unique_aero}")


Number of unique values in 'aero' column for 'desc' starting with 'boîte noire': 111


# ------------------ test sur A321_6216 ------------------

In [9]:

# Renommer les colonnes pour avoir le même nom de colonne dans chaque df
dataframes = [dfa, dfc, dfd, dflv]

for df in dataframes:
    if 'aero' not in df.columns:
        if 'ref_aero' in df.columns:
            df.rename(columns={'ref_aero': 'aero'}, inplace=True)
        elif 'linked_aero' in df.columns:
            df.rename(columns={'linked_aero': 'aero'}, inplace=True)
        elif 'aero_linked' in df.columns:
            df.rename(columns={'aero_linked': 'aero'}, inplace=True)
    if 'compo' not in df.columns:
        if 'ref_compo' in df.columns:
            df.rename(columns={'ref_compo': 'compo'}, inplace=True)
        elif 'compo_concerned' in df.columns:
            df.rename(columns={'compo_concerned': 'compo'}, inplace=True)

# Garder les lignes qui nous intéressent dans chaque df
dfa_A321_6216 = dfa[dfa['aero'] == "A321_6216"]
dfc_A321_6216 = dfc[dfc['aero'] == "A321_6216"]
dfd_A321_6216 = dfd[dfd['aero'] == "A321_6216"]
dflv_A321_6216 = dflv[dflv['aero'] == "A321_6216"]

common_columns = ['aero','compo']
# Merge les df à partir des colonnes communes
df_A320_2211 = pd.merge(dfc_A320_2211, dfd_A320_2211, on=common_columns)
df_A320_2211 = pd.merge(df_A320_2211, dfa_A320_2211, on='aero')
df_A320_2211 = pd.merge(df_A320_2211, dflv_A320_2211, on='aero')

In [11]:

# Renommer les colonnes pour avoir le même nom de colonne dans chaque df
dataframes = [dfa, dfc, dfd, dflv]

for df in dataframes:
    if 'aero' not in df.columns:
        if 'ref_aero' in df.columns:
            df.rename(columns={'ref_aero': 'aero'}, inplace=True)
        elif 'linked_aero' in df.columns:
            df.rename(columns={'linked_aero': 'aero'}, inplace=True)
        elif 'aero_linked' in df.columns:
            df.rename(columns={'aero_linked': 'aero'}, inplace=True)
    if 'compo' not in df.columns:
        if 'ref_compo' in df.columns:
            df.rename(columns={'ref_compo': 'compo'}, inplace=True)
        elif 'compo_concerned' in df.columns:
            df.rename(columns={'compo_concerned': 'compo'}, inplace=True)

# Garder les lignes qui nous intéressent dans chaque df
dfa_A330_5589 = dfa[dfa['aero'] == "A330_5589"]
dfc_A330_5589 = dfc[dfc['aero'] == "A330_5589"]
dfd_A330_5589 = dfd[dfd['aero'] == "A330_5589"]
dflv_A330_5589 = dflv[dflv['aero'] == "A330_5589"]

common_columns = ['aero','compo']
# Merge les df à partir des colonnes communes
df_A330_5589 = pd.merge(dfc_A330_5589, dfd_A330_5589, on=common_columns)
df_A330_5589 = pd.merge(df_A330_5589, dfa_A330_5589, on='aero')
df_A330_5589 = pd.merge(df_A330_5589, dflv_A330_5589, on='aero')

In [12]:
autopilote = df_A320_2211[df_A320_2211["desc"] == "Autopilote"]

print(autopilote.shape)
display(autopilote)

(152, 21)


,compo,aero,desc,lifespan,taux_usure_actuel,cout_composant,ref_deg,usure_cumulée,measure_day,need_replacement,...,debut_service,last_maint,en_maintenance,end_maint,date_diff,ref_vol,jour_vol,time_en_air,sensor_data,etat_voyant
741,AUT8-280,A320_2211,Autopilote,13009,22.22035,9064,D009057,28.07440,2023-11-16,False,...,2006-07-03,2023-10-24,False,NaT,NaN,V05845755,2023-11-07 14:13:00,11.8,"{'temp': '-1°C', 'pressure': '984 hPa', 'vibra...",1
742,AUT8-280,A320_2211,Autopilote,13009,22.22035,9064,D009057,28.07440,2023-11-16,False,...,2006-07-03,2023-10-24,False,NaT,NaN,V02246344,2023-11-12 14:13:00,11.4,"{'temp': '11°C', 'pressure': '972 hPa', 'vibra...",1
743,AUT8-280,A320_2211,Autopilote,13009,22.22035,9064,D009057,28.07440,2023-11-16,False,...,2006-07-03,2023-10-24,False,NaT,NaN,V08387796,2023-11-16 14:13:00,5.6,"{'temp': '-38°C', 'pressure': '918 hPa', 'vibr...",0
744,AUT8-280,A320_2211,Autopilote,13009,22.22035,9064,D009057,28.07440,2023-11-16,False,...,2006-07-03,2023-10-24,False,NaT,NaN,V01287468,2023-11-17 14:13:00,1.3,"{'temp': '43°C', 'pressure': '977 hPa', 'vibra...",2
745,AUT8-280,A320_2211,Autopilote,13009,22.22035,9064,D009057,28.07440,2023-11-16,False,...,2006-07-03,2023-10-24,False,NaT,NaN,V03910119,2023-11-18 14:13:00,0.9,"{'temp': '21°C', 'pressure': '900 hPa', 'vibra...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,AUT8-280,A320_2211,Autopilote,13009,22.22035,9064,D000044,53.44195,2023-12-25,False,...,2006-07-03,2023-10-24,False,NaT,NaN,V07902882,2023-12-19 14:13:00,8.1,"{'temp': '-1°C', 'pressure': '1096 hPa', 'vibr...",0
889,AUT8-280,A320_2211,Autopilote,13009,22.22035,9064,D000044,53.44195,2023-12-25,False,...,2006-07-03,2023-10-24,False,NaT,NaN,V02164760,2023-12-22 14:13:00,11.5,"{'temp': '-20°C', 'pressure': '919 hPa', 'vibr...",2
890,AUT8-280,A320_2211,Autopilote,13009,22.22035,9064,D000044,53.44195,2023-12-25,False,...,2006-07-03,2023-10-24,False,NaT,NaN,V02875933,2023-12-23 14:13:00,10.5,"{'temp': '21°C', 'pressure': '1090 hPa', 'vibr...",2
891,AUT8-280,A320_2211,Autopilote,13009,22.22035,9064,D000044,53.44195,2023-12-25,False,...,2006-07-03,2023-10-24,False,NaT,NaN,V06877572,2024-01-01 14:13:00,2.5,"{'temp': '48°C', 'pressure': '947 hPa', 'vibra...",0
